# Advent of Code 2021 (PowerShell)

## Day 1 - Sonar Sweep

[Puzzle Link](https://adventofcode.com/2021/day/1)

### Input

In [ ]:
[int[]]$Readings = Get-Content -Path ..\PuzzleInput\day1.txt

#### Learning Points

- By default, `Get-Content` will read the file line by line, creating an array of (string) lines.

- You need to compare the current and previous reading as numbers and not as strings.
Make PowerShell do the work by *casting* the variable as an array of integers, `[int[]]`.

### Puzzle 1

Using input file, count the number of times a depth measurement increases.

In [ ]:
# Direct variable assignment
$Increased = for ($Count = 1; $Count -le $Readings.Count; $Count++) {
    if ($Readings[$Count] -gt $Readings[$Count-1]) { 1 }
}
$Increased.Count

1451


In [ ]:
# Incrementing variable
$Increased = 0
for ($Count = 1; $Count -le $Readings.Count; $Count++) {
    if ($Readings[$Count] -gt $Readings[$Count-1]) { $Increased++ }
}
$Increased

1451


#### Learning Points

- You will need to use a loop to cycle through each of the readings.
I've chosen to use a `for` loop which allows me to initialize a counter, define the condition that ends the loop, and increment the counter.
  - You can increment or decrement an integer by using `++` or `--`.
- Instead of defining a variable that would contain the number of times the readings increased, I wanted to show you that you can assign the results of a `for` loops,
and many other loops and expressions, directly to a variable.
By doing so, I can output `1` and because of the variable assignment, `$Increased = for (...) { ... }`, it becomes an array of `1's`.
To get the answer, I use the array type property, `Count`.
- I've also included a second method which will likely take less memory and complete faster.

### Puzzle 2

Count the number of times the sum of measurements in this sliding window increases from the previous sum.

In [ ]:
$GroupCount = 0
$GroupIncreased = 0
do {
    $PreviousGroupSum = ($Readings[$GroupCount..($GroupCount+2)] | Measure-Object -Sum).Sum
    $CurrentGroupSum = ($Readings[($GroupCount+1)..($GroupCount+3)] | Measure-Object -Sum).Sum
    if ($CurrentGroupSum -gt $PreviousGroupSum) {
        $GroupIncreased++
    }
    $GroupCount++
} while ($GroupCount -lt $Readings.Count)
$GroupIncreased

1395


## Day 2 - Dive!

[Puzzle Link](https://adventofcode.com/2021/day/2)

### Input

In [ ]:
$SubMoves = Get-Content -Path ..\PuzzleInput\day2.txt

### Puzzle 1

What do you get if you multiply your final horizontal position by your final depth?

In [ ]:
$Position = 0
$Depth = 0
foreach ($Move in $SubMoves) {
    $Direction,[int]$Distance = $Move.Split(' ')
    switch ($Direction) {
        'forward'   { $Position += $Distance }
        'down'      { $Depth += $Distance }
        'up'        { $Depth -= $Distance }
    }
}
'Horizontal Position: {0}, Depth {1}, Location: {2}' -f $Position,$Depth,($Position * $Depth)

Horizontal Position: 1991, Depth 911, Location: 1813801


### Puzzle 2

With updated instructions, what do you get if you multiply your final horizontal position by your final depth?

In [ ]:
$Position = 0
$Depth = 0
$Aim = 0
foreach ($Move in $SubMoves) {
    $Direction,[int]$Distance = $Move.Split(' ')
    switch ($Direction) {
        'forward'   { 
            $Position += $Distance
            $Depth += $Distance * $Aim
        }
        'down'      { $Aim += $Distance }
        'up'        { $Aim -= $Distance }
    }
}
'Horizontal Position: {0}, Depth {1}, Location: {2}' -f $Position,$Depth,($Position * $Depth)

Horizontal Position: 1991, Depth 984716, Location: 1960569556
